In [15]:
# Read the beauty.csv.

# Implement the classification with various models and grid search cv.Ignore y column and create 3 groups using cluster techniques

import pandas as pd

beauty = pd.read_csv("C:\\Users\\HARSH\\OneDrive\\Desktop\\Bizmetric_Python\\datasets\\datasets\\beauty.csv")  
print(beauty)

       wage  exper  union  goodhlth  black  female  married  service  educ  \
0      5.73     30      0         1      0       1        1        1    14   
1      4.28     28      0         1      0       1        1        0    12   
2      7.96     35      0         1      0       1        0        0    10   
3     11.57     38      0         1      0       0        1        1    16   
4     11.42     27      0         1      0       0        1        0    16   
...     ...    ...    ...       ...    ...     ...      ...      ...   ...   
1255   1.61     25      0         1      1       1        0        1    12   
1256   1.68      4      0         1      0       1        1        1    12   
1257   3.29     35      0         1      1       1        0        1    12   
1258   2.31     15      0         1      1       1        1        1    10   
1259   1.92     24      0         0      0       1        0        1    16   

      looks  
0         4  
1         3  
2         4  
3      

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Feature Scaling

scaler = StandardScaler()
X_scaled = scaler.fit_transform(beauty)
X_scaled

array([[-0.12378548,  0.98619536, -0.61159284, ...,  1.62854963,
         0.54756497,  1.18942312],
       [-0.43502515,  0.81895361, -0.61159284, ..., -0.6140433 ,
        -0.21479068, -0.27127194],
       [ 0.35487966,  1.40429971, -0.61159284, ..., -0.6140433 ,
        -0.97714632,  1.18942312],
       ...,
       [-0.64752672,  1.40429971, -0.61159284, ...,  1.62854963,
        -0.21479068, -0.27127194],
       [-0.8578818 , -0.26811771, -0.61159284, ...,  1.62854963,
        -0.97714632, -0.27127194],
       [-0.94159454,  0.48447013, -0.61159284, ...,  1.62854963,
         1.30992061, -0.27127194]])

In [3]:
# Creating cluster labels (unsupervised learning)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)
clusters

array([2, 2, 2, ..., 2, 2, 0])

In [6]:
# Preparing dataset for classification

beauty['cluster'] = clusters
X = beauty.drop(columns=['cluster'])  # Features
y = beauty['cluster']                 # Labels

# Spliting the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Defining models and parameter grids

model_params = {
    "Logistic Regression": (
        LogisticRegression(max_iter=500),
        {
            "C": [0.01, 0.1, 1, 10],
            "solver": ["liblinear", "lbfgs"]
        }
    ),
    "Random Forest": (
        RandomForestClassifier(),
        {
            "n_estimators": [50, 100, 150],
            "max_depth": [5, 10, None]
        }
    ),
    "SVM": (
        SVC(),
        {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"]
        }
    )
}

In [14]:
# Train, tune and evaluating models using GridSearchCV

for name, (model, params) in model_params.items():
    print(f"\n Running GridSearchCV for {name}...")
    grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    
    print(f" Best Parameters: {grid.best_params_}")
    print(" Accuracy:", accuracy_score(y_test, y_pred))
    print(" Classification Report:\n", classification_report(y_test, y_pred))


 Running GridSearchCV for Logistic Regression...


C:\Users\HARSH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


 Best Parameters: {'C': 10, 'solver': 'lbfgs'}
 Accuracy: 0.9880952380952381
 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.98      0.99      0.99       130
           2       0.99      0.98      0.99       105

    accuracy                           0.99       252
   macro avg       0.99      0.99      0.99       252
weighted avg       0.99      0.99      0.99       252


 Running GridSearchCV for Random Forest...
 Best Parameters: {'max_depth': 10, 'n_estimators': 150}
 Accuracy: 0.9642857142857143
 Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       0.97      0.96      0.97       130
           2       0.95      0.96      0.96       105

    accuracy                           0.96       252
   macro avg       0.97      0.97      0.97       252
weighted avg       0.96      0.96  